In [28]:
from gurobipy import *
import numpy as np
import random
import time

n = 30  # nombre de sommets = len(V)
p = 3  # nombre de "facilities" = ressources
V = range(n)
#E = [(0, 1), (1, 2), (2, 3), (3, 4)]  # arêtes
#E = [(0, 3), (1, 2)]
E = []
proba = 0.75

# Note : un graphe peut ne pas être connexe
for i in range(n):
    for j in range(i+1,n):
        pp = random.random()
        if (pp < proba):
            E.append((i,j))
#print("Les arêtes du graphe :", E)
print("Nombre d'arêtes dans le graphe :", len(E))
        
F = range(p)  # F = {0, 1, ..., p-1}

def fct_w(n):
    return np.array([n - k for k in range(n)])

def fct_w_prime(w):
    n = len(w)
    w_prime = np.zeros(n)
    for k in range(n - 1):
        w_prime[k] = w[k] - w[k + 1]
    w_prime[n - 1] = w[n - 1]
    return w_prime

w = fct_w(n) # pour n = 5, w = [5,4,3,2,1], w_prime = [1,1,1,1,1]
w_prime = fct_w_prime(w)

# Bornes pour les agents et items
#lower_agent = [1] * n # li
#upper_agent = [m] * n # ui
#lower_item  = [1] * p # li_prime
#upper_item  = [1] * p # ui_prime

# Matrice des coûts aléatoires
c = np.random.randint(1, 1000, size=(n, p))
#c = np.array([[5,8,4,9,7],[1,3,2,7,8],[3,9,2,9,5],[10,1,3,3,4],[5,1,7,7,3]]) # Exemple du polycopié
#c = np.array([[1,3,3],[2,4,2],[3,2,3],[3,4,4],[1,3,2]])
#print("La matrice des couts c :")
#for i in range(n):
#    print("i =", i, "|", c[i])

Nombre d'arêtes dans le graphe : 35


In [29]:
def constr_model(c, w_prime, n, p):
    model = Model(name="vertex cover OWA")
    b = model.addVars(n, n, vtype=GRB.CONTINUOUS, name="b", lb=0)
    r = model.addVars(n, name='r', vtype=GRB.CONTINUOUS)
    x = model.addVars(n, p, vtype=GRB.CONTINUOUS, name="x", lb=0, ub=1)
    
    model.addConstrs((r[k1] + b[i, k1] >= sum(c[i, k] * x[i, k] for k in F)
                        for i in range(n) for k1 in range(n)), name='c3')
    # Pour chaque arête, et pour chaque ressource k, on impose x[i,k] + x[j,k] >= 1.
    for (i, j) in E:
        for k in F:
            model.addConstr(x[i, k] + x[j, k] >= 1, name="edge_{}_{}_{}".format(i, j, k))
    
    obj_fn = sum(w_prime[k1] * ((k1 + 1) * r[k1] + sum(b[i, k1] for i in range(n)))
                 for k1 in range(n))
    model.setObjective(obj_fn, GRB.MINIMIZE)
    model.Params.OutputFlag = 0
    return model, x

# Construction du modèle initial en LP relaxé.
start = time.time()
model, x = constr_model(c, w_prime, n, p)
#model.optimize()
#print("AVANT iterative_rounding", model.objVal)

def iterative_rounding(model, x, V, E, F):
    iteration = 0
    taille_E = len(E)
    taille_F = len(F)
    #somme_2 = [[0 for i in range(taille_E)] for k in range(taille_F)]
    iter_gurobi = 0
    it_frac = 0

    min_max_val = 1
    while True:
        # On résout la relaxation
        model.optimize()
        iter_gurobi += model.IterCount
        #print("JUSTE AU CAS OU", model.objVal)
        if model.Status != GRB.OPTIMAL:
            print("La résolution n'est pas optimale.")
            break
        
        #changed = False
        #sol = {}
        #for i in range(n):
        #    for k in F:
        #        sol[(i, k)] = x[i, k].X
                
        max_val = -1
        sel_i, sel_k = -1, -1
        for k in F:
            for i in V:
                if x[i, k].LB != x[i, k].UB: # donc on ne pourrait pas avoir 0 comme plus grande valeur
                    val = x[i, k].X
                    if val > max_val:
                        max_val = val
                        sel_i, sel_k = i, k
        #print("Valeur à fixer :", max_val, sel_i, sel_k)
        if max_val > 0 and max_val < min_max_val:
            min_max_val = max_val
    
        if sel_i == -1 or sel_k == -1 or max_val == 0: # en fait max_val = 0 signifie qu'on a déjà choisi tous les sommets à fixer et que par défaut, on ne prend pas les sommets restants
        # ca ne change rien donc à la solution optimale de fixer à 0 le UB des variables restantes, on peut gagner + de temps. 
        # en fait, contrairement à affectation, on a pas de borne sup, et fixer des valeurs 0 à 1 déteriorent le résultat.
            #print("Aucune variable trouvée, fin de l'algorithme")
            print("Nombre d'itérations prises par Gurobi :", iter_gurobi)
            break

        if max_val > 0 and max_val < 1:
            it_frac += 1

        x[sel_i, sel_k].lb = 1
        x[sel_i, sel_k].ub = 1

        # Pour chaque contrainte d'arête et pour chaque facility
        #for (i, j) in E:
        #    for k in F:
        #        # Si pour cette contrainte l'une des deux variables est déjà fixée à 1, on passe à l'itération suivante
        #        if abs(x[i, k].LB - 1.0) < 1e-5 or abs(x[j, k].LB - 1.0) < 1e-5:
        #            continue
        #
        #        xi_val = sol[(i, k)]
        #        xj_val = sol[(j, k)]
        #        # On fixe à 1 la variable dont la valeur est supérieure.
        #        if xi_val >= xj_val:
        #            print("Fixation de x[{}, {}] passant de {:.3f} à 1 (arête ({}, {}))".format(i, k, xi_val, i, j))
        #            x[i, k].LB = 1.0
        #            x[i, k].UB = 1.0
        #            changed = True
        #        else:
        #            print("Fixation de x[{}, {}] passant de {:.3f} à 1 (arête ({}, {}))".format(j, k, xj_val, i, j))
        #            x[j, k].LB = 1.0
        #            x[j, k].UB = 1.0
        #            changed = True

        model.update()

        # Pas obligé comme dans le problème d'affectation
        #indice_E = 0
        #for (i,j) in E:
        #    #indice_E += 1
        #    if i == sel_i:
        #        x[j, sel_k].UB = 0
        #    if j == sel_i:
        #        x[i, sel_k].UB = 0
                
        iteration += 1
        ## S'il n'y a eu aucun changement dans cette boucle, on sort du while.
        #if not changed:
        #    break
    #print("Arrondi itératif terminé après {} itérations.".format(iteration))
    return min_max_val, it_frac

# Lancement de l'arrondi itératif
min_max_val_o, it_frac_o = iterative_rounding(model, x, V, E, F)
end = time.time()
time_approx = end-start
val_approx = model.objVal

# Affichage de la solution finale
print('val_approx_o : %f' % val_approx)
print("min_max_val_o =", min_max_val_o)
print("time_approx(O) :", time_approx)
print("it_frac(O) =", it_frac_o)
#print("\nSolution finale :")
#for i in range(n):
#    for k in F:
#        print("x[{}, {}] = {:.3f}".format(i, k, x[i, k].X))

Nombre d'itérations prises par Gurobi : 597.0
val_approx_o : 132016.000000
min_max_val_o = 0.5
time_approx(O) : 0.05861043930053711
it_frac(O) = 42


In [30]:
#def sol_equitable_opti(c,w_prime,n,m,l_prime,u_prime,l,u,n,p):
#def sol_equitable_opti(c,w_prime,n,p,l_prime,u_prime):
def sol_equitable_opti(c,w_prime,n,p,E,F):
    opt_mod2 = Model(name = "vertex cover OWA")
    opt_mod2.Params.OutputFlag = 0
    
    b = opt_mod2.addVars(n, n, vtype = GRB.CONTINUOUS, name = "b", lb = 0)
    r = opt_mod2.addVars(n, name = 'r', vtype = GRB.CONTINUOUS)
    x = opt_mod2.addVars(n, p, vtype = GRB.BINARY, name = "x")
    
    #opt_mod2.addConstrs((l_prime[k] <= sum(x[i,k] for i in V) for k in F), name = 'c1a')
    #opt_mod2.addConstrs((sum(x[i,k] for i in V) <= u_prime[k] for k in F), name = 'c1b')
    #opt_mod2.addConstrs((l[i] <= sum(x[i,j] for j in range(m)) for i in range(n)), name = 'c2a')
    #opt_mod2.addConstrs((sum(x[i,j] for j in range(m)) <= u[i] for i in range(n)), name = 'c2b')
    
    opt_mod2.addConstrs((r[k1] + b[i,k1] >= sum(c[i,k]*x[i,k] for k in range(p)) for i in range(n) for k1 in range(n)), name = 'c3')
    for (i, j) in E:
        for k in F:
            opt_mod2.addConstr(x[i, k] + x[j, k] >= 1)
    
    obj_fn2 = sum(w_prime[k1]*((k1+1)*r[k1] + sum(b[i,k1] for i in range(n))) for k1 in range(n))
    opt_mod2.setObjective(obj_fn2, GRB.MINIMIZE)

    return opt_mod2, x

#start = time.time()
opt_mod2, x1 = sol_equitable_opti(c,w_prime,n,p,E,F)
opt_mod2.optimize()
#print("Nombre d'itérations :", opt_mod2.IterCount)
#print("Temps de résolution (s) :", opt_mod2.Runtime)
#end = time.time()
#time_exact = end-start
val_exact = opt_mod2.objVal
print('val_exact_o : %f' % val_exact)
#print("time_exact(O) :", time_exact)

#print("\nSolution finale :")
#for i in range(n):
#    for k in F:
#        print("x[{}, {}] = {:.3f}".format(i, k, x[i, k].X))

val_exact_o : 127793.000000


In [31]:
def chassein_algo(c,w,n,p,E,F):
    opt_mod2 = Model(name = "C-MIP")
    opt_mod2.Params.OutputFlag = 0

    y = opt_mod2.addVars(n, vtype = GRB.CONTINUOUS, name = "y")
    #b = opt_mod2.addVars(n, n, vtype = GRB.CONTINUOUS, name = "b", lb = 0)
    #r = opt_mod2.addVars(n, name = 'r', vtype = GRB.CONTINUOUS)
    alpha = opt_mod2.addVars(n, name = 'alpha', vtype = GRB.CONTINUOUS)
    beta = opt_mod2.addVars(n, name = 'beta', vtype = GRB.CONTINUOUS)
    x = opt_mod2.addVars(n, p, vtype = GRB.BINARY, name = "x")
    
    #opt_mod2.addConstrs((l_prime[j] <= sum(x[i,j] for i in range(n)) for j in range(m)), name = 'c1a')
    #opt_mod2.addConstrs((sum(x[i,j] for i in range(n)) <= u_prime[j] for j in range(m)), name = 'c1b')
    #opt_mod2.addConstrs((l[i] <= sum(x[i,j] for j in range(m)) for i in range(n)), name = 'c2a')
    #opt_mod2.addConstrs((sum(x[i,j] for j in range(m)) <= u[i] for i in range(n)), name = 'c2b

    opt_mod2.addConstrs((y[i] == sum(c[i,k]*x[i,k] for k in range(p)) for i in range(n)), name = 'c1')
    opt_mod2.addConstrs((alpha[i] + beta[j] >= w[j]*y[i] for i in range(n) for j in range(n)), name = 'c2')
    #opt_mod2.addConstrs((r[k] + b[i,k] >= sum(c[i,j]*x[i,j] for j in range(m)) for i in range(n) for k in range(n)), name = 'c3')

    for (i, j) in E:
        for k in F:
            opt_mod2.addConstr(x[i, k] + x[j, k] >= 1)
    #w_prime = fct_w_prime(w)
    
    #obj_fn2 = quicksum(w_prime[k]*((k+1)*r[k] + quicksum(b[i,k] for i in range(n))) for k in range(n))
    obj_fn2 = quicksum(alpha[i] + beta[i] for i in range(n))
    opt_mod2.setObjective(obj_fn2, GRB.MINIMIZE)

    return opt_mod2, x

#start = time.time()
opt_mod2, x1 = chassein_algo(c,w,n,p,E,F)
opt_mod2.optimize()
#print("Nombre d'itérations :", opt_mod2.IterCount)
#print("Temps de résolution (s) :", opt_mod2.Runtime)
#end = time.time()
#time_exact_c = end-start
#print("time_exact(C) :", time_exact_c)
##opt_mod2.write("sol_exacte_poly.lp")

mod_chassein = opt_mod2.objVal
print('val_exact_c : %f' % mod_chassein)

val_exact_c : 127793.000000


In [32]:
def chassein_relaxation(c,w,n,p,E,F):
    opt_mod2 = Model(name = "C-MIP")
    opt_mod2.Params.OutputFlag = 0

    y = opt_mod2.addVars(n, vtype = GRB.CONTINUOUS, name = "y")
    #b = opt_mod2.addVars(n, n, vtype = GRB.CONTINUOUS, name = "b", lb = 0)
    #r = opt_mod2.addVars(n, name = 'r', vtype = GRB.CONTINUOUS)
    alpha = opt_mod2.addVars(n, name = 'alpha', vtype = GRB.CONTINUOUS)
    beta = opt_mod2.addVars(n, name = 'beta', vtype = GRB.CONTINUOUS)
    #x = opt_mod2.addVars(n, p, vtype = GRB.BINARY, name = "x")
    x = opt_mod2.addVars(n, p, vtype=GRB.CONTINUOUS, name="x", lb=0, ub=1)
    
    #opt_mod2.addConstrs((l_prime[j] <= sum(x[i,j] for i in range(n)) for j in range(m)), name = 'c1a')
    #opt_mod2.addConstrs((sum(x[i,j] for i in range(n)) <= u_prime[j] for j in range(m)), name = 'c1b')
    #opt_mod2.addConstrs((l[i] <= sum(x[i,j] for j in range(m)) for i in range(n)), name = 'c2a')
    #opt_mod2.addConstrs((sum(x[i,j] for j in range(m)) <= u[i] for i in range(n)), name = 'c2b

    opt_mod2.addConstrs((y[i] == sum(c[i,k]*x[i,k] for k in range(p)) for i in range(n)), name = 'c1')
    opt_mod2.addConstrs((alpha[i] + beta[j] >= w[j]*y[i] for i in range(n) for j in range(n)), name = 'c2')
    #opt_mod2.addConstrs((r[k] + b[i,k] >= sum(c[i,j]*x[i,j] for j in range(m)) for i in range(n) for k in range(n)), name = 'c3')

    for (i, j) in E:
        for k in F:
            opt_mod2.addConstr(x[i, k] + x[j, k] >= 1)
    #w_prime = fct_w_prime(w)
    
    #obj_fn2 = quicksum(w_prime[k]*((k+1)*r[k] + quicksum(b[i,k] for i in range(n))) for k in range(n))
    obj_fn2 = quicksum(alpha[i] + beta[i] for i in range(n))
    opt_mod2.setObjective(obj_fn2, GRB.MINIMIZE)

    return opt_mod2, x

# Construction du modèle initial en LP relaxé.
start = time.time()
model2, x2 = chassein_relaxation(c,w,n,p,E,F)
#model2.optimize()
#print("AVANT iterative_rounding", model2.objVal)

# Lancement de l'arrondi itératif
min_max_val_c, it_frac_c = iterative_rounding(model2, x2, V, E, F)
end = time.time()
time_approx_chassein = end-start
val_approx_chassein = model2.objVal

# Affichage de la solution finale
print('val_approx_c : %f' % val_approx_chassein)
print("min_max_val_c :", min_max_val_c)
print("time_approx(C) :", time_approx_chassein)
print("it_frac(C) =", it_frac_c)

Nombre d'itérations prises par Gurobi : 657.0
val_approx_c : 132016.000000
min_max_val_c : 0.5
time_approx(C) : 0.07829093933105469
it_frac(C) = 42


In [33]:
from itertools import combinations

def constr_3e_model(V, E, F, c, w_prime):
    model = Model("L-Dominance Vertex Cover")
    model.Params.OutputFlag = 0

    x_vc = model.addVars(V, F, vtype=GRB.CONTINUOUS, lb = 0, ub = 1, name="x_vc")     # vertex cover
    x_obj = model.addVars(V, F, vtype=GRB.CONTINUOUS, lb = 0, ub = 1, name="x_obj")   # Pour le calcul de z
    z = model.addVars(V, vtype=GRB.CONTINUOUS, name="z")
    l = model.addVars(V, vtype=GRB.CONTINUOUS, name="l")

    sync_constr = {}
    for i in V:
        for k in F:
            sync_constr[i, k] = model.addConstr(x_vc[i, k] == x_obj[i, k], name=f"sync_{i}_{k}")

    for (i, j) in E:
        for k in F:
            model.addConstr(x_vc[i, k] + x_vc[j, k] >= 1, name=f"vc_{i}_{j}_{k}")

    for i in V:
        model.addConstr(z[i] == quicksum(c[i][k] * x_obj[i, k] for k in F), name=f"z_{i}")

    #seen_constr = set()
    #perms = list(permutations(V))
    
    #for pi in list(permutations(V)):
    #    for j in V:
    #        key = tuple(sorted(pi[:j+1]))
    #
    #        if key not in seen_constr:
    #            expr = quicksum(z[i] for i in pi[:j+1])
    #            model.addConstr(l[j] >= expr, name=f"l_{j}")
    #            seen_constr.add(key)

    vector = [i for i in range(n)]
    for j in V:
        combinaisons = list(combinations(vector, j+1))
        for comb in combinaisons:
            expr = quicksum(z[elem_comb] for elem_comb in comb)
            model.addConstr(l[j] >= expr, name=f"l_{j}")
            #seen_constr.add(key)

    model.setObjective(quicksum(w_prime[i] * l[i] for i in V), GRB.MINIMIZE)

    model.update()

    return model, x_vc, x_obj, sync_constr #, z, l

In [34]:
def iterative_rounding_3e_mod(model, x_vc, x_obj, sync_constr, V, E, F):
    iteration = 0
    iter_gurobi = 0
    it_frac_3e = 0

    while True:
        model.optimize()
        iter_gurobi += model.IterCount

        if model.Status != GRB.OPTIMAL:
            print("Résolution non optimale.")
            break

        max_val = -1
        sel_i, sel_k = -1, -1

        for k in F:
            for i in V:
                if x_vc[i, k].LB != x_vc[i, k].UB:  # encore libre
                    val = x_vc[i, k].X
                    if val > max_val:
                        max_val = val
                        sel_i, sel_k = i, k
        #print(max_val, sel_i, sel_k)

        if sel_i == -1 or sel_k == -1 or max_val <= 1e-5:
            print("Terminé. Nombre d'itérations Gurobi total :", iter_gurobi)
            print("Nombre tot d'iterations :", iteration)
            break

        if max_val > 0 and max_val < 1:
            it_frac_3e += 1

        model.remove(sync_constr[sel_i, sel_k])

        x_vc[sel_i, sel_k].LB = 1.0
        x_vc[sel_i, sel_k].UB = 1.0

        x_obj[sel_i, sel_k].LB = 0.5
        x_obj[sel_i, sel_k].UB = 0.5

        model.update()
        iteration += 1
    return it_frac_3e

In [35]:
#start = time.time()
#model_test, x_vc, x_obj, sync_constr = constr_3e_model(V,E,F,c,w_prime)
#model_test.write("3e_mod.lp")
#it_frac_3e = iterative_rounding_3e_mod(model_test, x_vc, x_obj, sync_constr, V, E, F)
#end = time.time()
#val_new_mod = model_test.objVal
#time_approx_3e_mod = end-start
#model_test.write("3e_mod_end.lp")

# Affichage de la solution finale
#print('val_new_mod : %f' % val_new_mod)
#val_new_mod2 = 2*val_new_mod
#print('2*val_new_mod : %f' % val_new_mod2)
#print("time_approx_3e_mod :", time_approx_3e_mod)

#print("\nSolution finale :")
#for i in range(n):
#    for k in F:
#        print("x[{}, {}] = {:.3f}".format(i, k, x_vc[i, k].X))

In [36]:
def new_model(V, E, F, c, w_prime):
    model = Model("L-Dominance Vertex Cover")
    model.Params.OutputFlag = 0

    x_vc = model.addVars(V, F, vtype=GRB.BINARY, lb = 0, ub = 1, name="x_vc")     # vertex cover
    #x_obj = model.addVars(V, F, vtype=GRB.BINARY, lb = 0, ub = 1, name="x_obj")   # Pour le calcul de z
    z = model.addVars(V, vtype=GRB.CONTINUOUS, name="z")
    l = model.addVars(V, vtype=GRB.CONTINUOUS, name="l")

    #sync_constr = {}
    #for i in V:
    #    for k in F:
    #        sync_constr[i, k] = model.addConstr(x_vc[i, k] == x_obj[i, k], name=f"sync_{i}_{k}")

    for (i, j) in E:
        for k in F:
            model.addConstr(x_vc[i, k] + x_vc[j, k] >= 1, name=f"vc_{i}_{j}_{k}")

    for i in V:
        model.addConstr(z[i] == quicksum(c[i][k] * x_vc[i, k] for k in F), name=f"z_{i}")

    #seen_constr = set()
    #perms = list(permutations(V))
    
    #for pi in list(permutations(V)):
    #    for j in V:
    #        key = tuple(sorted(pi[:j+1]))
    #
    #        if key not in seen_constr:
    #            expr = quicksum(z[i] for i in pi[:j+1])
    #            model.addConstr(l[j] >= expr, name=f"l_{j}")
    #            seen_constr.add(key)

    vector = [i for i in range(n)]
    for j in V:
        combinaisons = list(combinations(vector, j+1))
        for comb in combinaisons:
            expr = quicksum(z[elem_comb] for elem_comb in comb)
            model.addConstr(l[j] >= expr, name=f"l_{j}")
            #seen_constr.add(key)

    model.setObjective(quicksum(w_prime[i] * l[i] for i in V), GRB.MINIMIZE)

    model.update()

    return model, x_vc #, z, l

#start = time.time()
#opt_mod3, x1 = new_model(V, E, F, c, w_prime)
#opt_mod3.optimize()
#end = time.time()
#time_exact_3e = end-start
#print("time_exact_3e :", time_exact_3e)

#val_exact_3e = opt_mod3.objVal
#print('val_exact_3e : %f' % val_exact_3e)

val_exact_3e : 127793.000000


In [37]:
def constr_model_dynamic(V, E, F, c, w_prime):
    model = Model("L-Dominance Vertex Cover")
    model.Params.OutputFlag = 0

    # Variables
    x_vc = model.addVars(V, F, vtype=GRB.CONTINUOUS, lb=0, ub=1, name="x_vc")
    x_obj = model.addVars(V, F, vtype=GRB.CONTINUOUS, lb=0, ub=1, name="x_obj")
    z = model.addVars(V, vtype=GRB.CONTINUOUS, name="z")
    # l[0] = l1, l[1] = l2, etc
    l = model.addVars(V, vtype=GRB.CONTINUOUS, name="l")

    sync_constr = {}
    for i in V:
        for k in F:
            sync_constr[i, k] = model.addConstr(x_vc[i, k] == x_obj[i, k], name=f"sync_{i}_{k}")

    for (i, j) in E:
        for k in F:
            model.addConstr(x_vc[i, k] + x_vc[j, k] >= 1, name=f"vc_{i}_{j}_{k}")

    for i in V:
        model.addConstr(z[i] == quicksum(c[i][k] * x_obj[i, k] for k in F), name=f"z_{i}")

    model.setObjective(quicksum(w_prime[i] * l[i] for i in V), GRB.MINIMIZE)
    model.update()
    return model, x_vc, x_obj, z, l, sync_constr

def iterative_rounding_dynamic_sorted(model, x_vc, x_obj, z, l, sync_constr, V, E, F, tol=1e-6):
    iteration = 0
    total_gurobi_iters = 0
    min_max_val = 1
    it_frac_3e = 0

    while True:
        model.optimize()
        total_gurobi_iters += model.IterCount

        if model.Status != GRB.OPTIMAL:
            print("Résolution non optimale.")
            break

        z_vals = [(i, z[i].X) for i in V]
        sorted_z = sorted(z_vals, key=lambda tup: tup[1], reverse=True) # on trie en fonction du 2eme element du couple

        prefix_sum = 0
        violation_found = False
        # Pour k de 0 à n-1 correspond à la vérification sur les k+1 plus grandes valeurs
        for k, (i_val, val) in enumerate(sorted_z):
            prefix_sum += val
            if l[k].X < prefix_sum - tol:
                expr = quicksum(z[j] for j, _ in sorted_z[:k+1]) # NOTE : on est obligé d'utiliser z et pas z_sorted, car ce sont des variables
                # Or, à la prochaine itération, on aura un nouveau z et donc un autre sorted_z, d'ou l'intéret de garder z, sinon, ca ne marchera pas !
                model.addConstr(l[k] >= expr, name=f"l_sorted_{k}")
                violation_found = True
        if violation_found:
            model.update()
            continue  # on relance l'optimisation pour tenir compte des nouvelles contraintes (donc on saute le reste)

        max_val = -1
        sel_i, sel_k = -1, -1

        for k in F:
            for i in V:
                if x_vc[i, k].LB != x_vc[i, k].UB:  # variable non fixée
                    val = x_vc[i, k].X
                    if val > max_val:
                        max_val = val
                        sel_i, sel_k = i, k

        if max_val > 0 and max_val < min_max_val:
            min_max_val = max_val

        if sel_i == -1 or sel_k == -1 or max_val <= 1e-5: # tolerance
            print("Terminé. Nombre total d'itérations Gurobi:", total_gurobi_iters)
            #print("Nombre tot d'iterations :", iteration)
            break

        #print(max_val, sel_i, sel_k)
        if max_val > 0 and max_val < 1:
            it_frac_3e += 1

        # On enlève la contrainte de synchronisation pour la variable fixée
        model.remove(sync_constr[sel_i, sel_k])

        x_vc[sel_i, sel_k].LB = 1.0
        x_vc[sel_i, sel_k].UB = 1.0

        x_obj[sel_i, sel_k].LB = 0.5
        x_obj[sel_i, sel_k].UB = 0.5

        model.update()
        iteration += 1

    return min_max_val, it_frac_3e

start = time.time()
model_dyn, x_vc, x_obj, z, l, sync_constr = constr_model_dynamic(V, E, F, c, w_prime)
#model_dyn.write("modele_dynamique.lp")
min_max_val_3e_v2, it_frac_3e = iterative_rounding_dynamic_sorted(model_dyn, x_vc, x_obj, z, l, sync_constr, V, E, F)
end = time.time()
val_new_mod_v2 = model_dyn.objVal
#model_dyn.write("modele_dynamique_end.lp")
time_approx_3e_v2 = end-start
print("time_approx_3e_v2 :", time_approx_3e_v2)
print("min_max_val_3e_v2 :", min_max_val_3e_v2)

# Affichage de la solution finale
print("Valeur de l'objectif:", val_new_mod_v2)
val_new_mod_v2_2 = 2*val_new_mod_v2
print('2*val_new_mod : %f' % val_new_mod_v2_2)
print("it_frac(3e) =", it_frac_3e)

0.5 0 0
0.5 1 0
0.5 2 0
0.5 3 0
0.5309017223910878 4 0
0.5309017223910841 5 0
0.7326607818411099 8 0
1.0 9 0
0.5 0 1
0.5 1 1
0.5 2 1
0.5 3 1
0.5 4 1
0.5 5 1
0.5235235235235236 6 1
0.81957928802589 9 1
0.5856595511767927 0 4
0.5904255319148937 2 4
0.651623663298342 4 4
0.6516236632983421 5 4
0.6516236632983415 6 4
0.7358472258622099 7 4
0.7729757761747555 8 4
0.9769268618217988 9 4
0.6842105263157895 0 2
0.7261380323054332 1 2
0.7874015748031493 3 2
0.7883254716981132 4 2
0.7883254716981132 5 2
0.7883254716981132 7 2
0.8909299655568219 8 2
1.0 9 2
0.5470701248799255 4 3
0.5470701248799231 1 3
0.6472392638036811 6 3
0.6494538232373386 7 3
0.8647887323943657 8 3
0.8687580025608195 9 3
0.6060903732809431 2 3
0.5 0 5
0.5 1 5
0.5 2 5
0.6210191082802548 3 5
0.6334745762711864 4 5
0.6334745762711864 5 5
0.6376518218623481 6 5
1.0 7 5
1.0 8 5
Terminé. Nombre total d'itérations Gurobi: 2735.0
time_approx_3e_v2 : 0.24094867706298828
min_max_val_3e_v2 : 0.5
Valeur de l'objectif: 65383.5
2*val_new_

In [38]:
ratio_o = val_approx/val_exact
print("O-MIP, Ratio de :", ratio_o)
ratio_c = val_approx_chassein/val_exact
#ratio_c = val_approx_chassein/mod_chassein
print("C-MIP, Ratio de :", ratio_c)
#ratio_3e = val_new_mod2/val_exact_3e
#ratio_3e = val_new_mod2/val_exact
#print("New model, Ratio de :", ratio_3e)
ratio_3e_v2 = val_new_mod_v2_2/val_exact
print("new mod avec problème de séparation, Ratio de :", ratio_3e_v2)
print("------")

O-MIP, Ratio de : 1.033045628477303
C-MIP, Ratio de : 1.033045628477303
new mod avec problème de séparation, Ratio de : 1.0232720102040018
------
